In [1]:
import cv2
import numpy as np
import os
from glob import glob

In [2]:
print(os.getcwd())

C:\Users\hi\Desktop\DeepLearning


In [3]:
base_dir = os.chdir(os.getcwd() + "/img-fashion")
print(os.getcwd())

C:\Users\hi\Desktop\DeepLearning\img-fashion


In [4]:
data_list = glob('*.jpg')
print('파일 개수 >> ', len(data_list))
print('파일 목록 타입 >> ', type(data_list))

파일 개수 >>  7652
파일 목록 타입 >>  <class 'list'>


In [5]:
data_list[:3]

['blouse (1).jpg', 'blouse (10).jpg', 'blouse (100).jpg']

In [6]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [7]:
token = text_to_word_sequence(data_list[0])
print(token)

['blouse', '1', 'jpg']


In [8]:
label = []
for x in data_list:
    token = text_to_word_sequence(x)
    label.append(token[0])
print(len(label))

7652


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
items = label
encoder = LabelEncoder()
label = encoder.fit_transform(items)
label 

array([0, 0, 0, ..., 7, 7, 7], dtype=int64)

In [11]:
from matplotlib import pyplot as plt

In [12]:
# 이미지마다 크기가 다 다르다.-> 픽셀 사이즈 동일해게 해주어야 전처리가 가능하다.
data_height = 150
data_width = 150
channel_n = 3

In [13]:
img_list = []
for x in data_list:
    imread_image = cv2.imread(x)/255
    resized_one = cv2.resize(imread_image, (data_height, data_width))
    img_list.append(resized_one)
print(len(img_list))
print(img_list[0].shape)

7652
(150, 150, 3)


In [14]:
def make_file(dat_height, data_width, channel_n, data_list_len):
    label = []
    images = np.zeros((data_list_len, dat_height, data_width, channel_n))
    
    for i, image in enumerate(data_list):
        
        token = text_to_word_sequence(image)
        label.append(token[0])
        
        image = cv2.imread(image)
        image = cv2.resize(image, (dat_height, data_width)) / 255
        
        images[i, :, :, :] = image
    label = np.array(label)
    return (label, images)    

In [15]:
label, images = make_file(data_height, data_width, channel_n, len(data_list))

In [16]:
label

array(['blouse', 'blouse', 'blouse', ..., 'tshirt', 'tshirt', 'tshirt'],
      dtype='<U7')

In [17]:
items = label
encoder = LabelEncoder()
label = encoder.fit_transform(items)
label, len(label) 

(array([0, 0, 0, ..., 7, 7, 7], dtype=int64), 7652)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x = images
y = label

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size= 0.2,
                                                    random_state=55
                                                   )
print(X_train.shape, X_test.shape)

(6121, 150, 150, 3) (1531, 150, 150, 3)


In [21]:
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential

In [26]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(data_height, data_width, 3)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(8, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 17, 17, 64)      

In [27]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)

In [28]:
es = EarlyStopping(patience=3, restore_best_weights=True)
cp = ModelCheckpoint('fashion_cnn_1.h5')

In [ ]:
cnn_history = model.fit( X_train, 
                         y_train,                
                         batch_size=128,
                         epochs=10,
                         validation_data = (X_test, y_test),
                         callbacks = [es, cp],
                         verbose=2
                        )

Epoch 1/10


In [ ]:
test_result = model.evaluate(X_test, y_test)
test_result

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1) ## 1행2열의 도표, 1열
plt.plot(cnn_history.history['accuracy'])
plt.plot(cnn_history.history['val_accuracy'])
plt.title('model acc')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.legend(['train_acc','test_acc'])

plt.subplot(1, 2, 2) ## 1행2열의 도표, 2열
plt.plot(cnn_history.history['loss'])
plt.plot(cnn_history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train_loss','test_loss'])